# Setup environment

In [17]:
# Essentials
import os
import numpy as np
import importlib

# PyTorch
import torch
from torch.utils.data import DataLoader

# Utils
import utils
importlib.reload(utils)

from utils import get_dict, get_files, check_corrupted_imgs, perturb_image, save_image, make_dirs, get_model, select_gpu
from utils import CustomTransforms, My_data, FocalLoss

# OnePixelAttack
import OnePixelAttack

importlib.reload(OnePixelAttack)

# TriangleAttack
import TriangleAttack
importlib.reload(TriangleAttack)

<module 'TriangleAttack' from '/home/jovyan/TriangleAttack.py'>

## Setup Cuda

In [18]:
# Set a higher max split size to avoid memory problems
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [19]:
global device
device = torch.device(f"cuda:{select_gpu()}" if torch.cuda.is_available() else "cpu")

print(device)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=None, abbreviated=False))
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

Selecting GPU 2 with 14903MB free memory
cuda:2
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  109982 KB |  109982 KB |  109982 KB |       0 B  |
|       from large pool |   99392 KB |   99392 KB |   99392 KB |       0 B  |
|       from small pool |   10590 KB |   10590 KB |   10590 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |  109982 KB |  109982 KB |  109982 KB |       0 B  |
|       from lar

## Get the data

In [20]:
# Load train and test files that are used for the model.
train_dict = get_dict("train.txt")
print("done")
test_dict = get_dict("test.txt")
print("done")

print(check_corrupted_imgs(train_dict, test_dict))

print("copying completed")

Opening Train.txt
done
Opening Train.txt
done
True
copying completed


In [21]:
train_files = get_files("./dataset/train/**/**/*.png")

test_files = get_files("./dataset/test/**/**/*.png")

# print(train_files)

## Setup Transformers

In [22]:
custom_transforms = CustomTransforms()
resize_transform = custom_transforms.get_transform("resize_tensor")
test_transform = custom_transforms.get_transform("test")

# Random Attack 

In [23]:
# original = My_data([test_files[0]])[0][0]

# p_tensor = np.copy(original)

# for i in range(100):
#     perturbation = [i, i] + list(np.random.choice(range(256), size=3))

#     p_tensor = perturb_image(perturbation, p_tensor)

# save_image(p_tensor, "perturbedimagerandom.png")


In [24]:
# tensor_list = [original, p_tensor]

# org = resize_transform(image=original)
# org_im = org['image']

# org_norm = test_transform(image=original)
# org_norm_im = org_norm['image']

# save_image(org_im, "org.png")
# save_image(org_norm_im, "org_norm.png")

In [25]:
# per = resize_transform(image=p_tensor)
# per_im = per['image']

# per_norm = test_transform(image=p_tensor)
# per_norm_im = per_norm['image']

# save_image(per_im, "per.png")
# save_image(per_norm_im, "per_norm.png")

In [26]:
# Deze wordt nu niet gecallt om tijd te besparen tijdens run all (nodig na veranderen geimporteerd bestand)
def simple_one_pixel_attack():
    make_dirs("simple_one_pixel")
    orgs = My_data(test_files)
    for i in range(orgs.__len__()):
        p_tensor = np.copy(orgs.__getitem__(i)[0])
        for j in range(200):
            random_x = np.random.choice(range(1, 698))
            random_y = np.random.choice(range(1, 398))

            random_xs = [random_x - 1, random_x, random_x + 1]
            random_ys = [random_y - 1, random_y, random_y + 1]

            for x in random_xs:
                for y in random_ys:
                    perturbation = [y, x] + list(np.random.choice(range(256), size=3))
                    p_tensor = perturb_image(perturbation, p_tensor)
                    
        save_image(p_tensor, os.path.join(os.getcwd(), "dataset", "simple_one_pixel", orgs.__getclass__(i), orgs.__getzoom__(i), orgs.__getname__(i)))


In [27]:
# org_norm = My_data(test_files, transforms=test_transform)
# perturb_data = My_data(get_files("./dataset/test_onepixel/**/**/*.png"), transforms=test_transform)

# org_dataloader = DataLoader(org_norm)
# pertrubed_dataloader = DataLoader(perturb_data)

## Test performance of Simple OnePixel Attack

In [28]:
# model = get_model(device, test_dict)

# model.eval()
# correct_org = 0
# correct_pert = 0
# confs_org = []
# confs_pert = []
# with torch.no_grad():
#     print("Testing network without attacks...")
#     for i, (inputs, labels) in enumerate(org_dataloader):
#         inputs = inputs.to(device)
#         labels = labels.float()
#         labels = labels.to(device)
#         outputs = model(inputs)
#         confs_org.append(outputs)
        
#         if torch.argmax(outputs) == torch.argmax(labels):
#             correct_org += 1
#     print(f"Accuracy of network without attack: {correct_org/len(test_files)}")
# # ######################## Run TinySwin without attacks ########################
#     print("Testing network with OnePixel attack...")
#     for i, (inputs, labels) in enumerate(pertrubed_dataloader):
#         inputs = inputs.to(device)
#         labels = labels.float()
#         labels = labels.to(device)
#         outputs = model(inputs)
#         confs_pert.append(outputs)
        
#         if torch.argmax(outputs) == torch.argmax(labels):
#             correct_pert += 1
#     print(f"Accuracy of network with OnePixel attack: {correct_pert/len(test_files)}")

#     # Compare the performance in confidence outputs
#     print(sum([torch.max(confs_org[i]) > torch.max(confs_pert[i]) for i in range(len(test_files))]), len(test_files))

In [29]:
# # Show all differences in predictions
# for i in range(len(test_files)):
#     if torch.argmax(confs_org[i]) != torch.argmax(confs_pert[i]):
#         print("Different predictions")
#         print(confs_org[i])
#         print(confs_pert[i])
#         print(test_files[i])

# Advanced OnePixel Attack

In [30]:
# org_norm = My_data(test_files, transforms=test_transform)
# org_dataloader = DataLoader(org_norm)

# model = get_model(device, test_dict, "resnet")

# model.eval()
# with torch.no_grad():
#     success = 0
#     correct_init_classification = 0
#     for i, (image, label) in enumerate(org_dataloader):
#         success, correct_init_classification = OnePixelAttack.attack(i, model, device, image, label, pixel_count=1, maxiter=50, popsize=15)
#         success += success  
#         correct_init_classification += correct_init_classification


#     print(success)
#     print("Accuracy", correct_init_classification / len(org_dataloader.dataset))

# Triangle attack   

In [32]:
import os
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "/usr/bin/git"

import git
git.refresh("/usr/bin/git")

from foolbox import PyTorchModel
from TriangleAttack import TA


org_norm = My_data(test_files, transforms=test_transform)
org_dataloader = DataLoader(org_norm)
model = PyTorchModel(get_model(device, test_dict).eval(), bounds=(0,1), device=device)

In [33]:
# run on a10 gpu as it has highest memory
print("Attack !")

with torch.no_grad():
    ta_model = TA(model, input_device=device)
    my_advs, q_list, my_intermediates, max_length = ta_model.attack(org_dataloader)
    print('TA Attack Done')

Attack !


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.28 GiB (GPU 2; 14.56 GiB total capacity; 12.07 GiB already allocated; 976.56 MiB free; 12.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Test Area

In [ ]:
# import albumentations
# import albumentations.pytorch

# transforms_a =  albumentations.Compose([
#                 albumentations.Resize(256, 256),
#                 albumentations.Normalize(mean=[0, 0, 0], std=[255, 255, 255], max_pixel_value=1.0),
#                 albumentations.pytorch.transforms.ToTensorV2()
#             ])

# test_a= My_data([test_files[0]], transforms_a)

# save_image(test_a.__getitem__(0)[0], "attemptofsucces_a.png")

# transforms_b =  albumentations.Compose([
#                 albumentations.Resize(256, 256),
#                 albumentations.Normalize(mean=[0 , 0, 0], std=[1, 1, 1], max_pixel_value=255),
#                 albumentations.pytorch.transforms.ToTensorV2()
#             ])

# test_b = My_data([test_files[0]], transforms_b)

# save_image(test_b.__getitem__(0)[0], "attemptofsucces_b.png")

In [ ]:
# print(org_dataloader.dataset)